# A5 Statistical Analyses of Clinical Datasets

#### Welcome to A5! Please enter answers to the questions in the specified Markdown cells below, and complete the code snippets in the associated Python files as specified. When you are done with the assignment, follow the instructions at the end of this assignment to submit.

### Learning Objective 🌱
In this assignment, you will work with a dataset that we have prepared for you using a process similar to what you did in A3 and A4. The dataset describes patients from the [MIMIC III database](https://mimic.physionet.org/mimictables/patients/) who were put on mechanical ventilation and were stable for 12 hours. Some of these patients then experienced a sudden and sustained drop in oxygenation, while others did not. You will practice using common time-saving tools in the **Pandas 🐼** library, the **NumPy 🔢** library, the **sklearn 🔬** library, and **Python 🐍** programming language that are ideally suited to these tasks.

### Resources 📖
- Pandas Cheat Sheet 🐼: [https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)



### Environment Set-Up 🐍
To begin, we will need to set up an virtual environment with the necessary packages. A virtual environment is a self-contained directory that contains a Python interpreter (aka Python installation) and any additional packages/modules that are required for a specific project. It allows you to isolate your project's dependencies from other projects that may have different versions or requirements of the same packages.

In this course, we require that you utilize [Miniconda](https://docs.conda.io/en/latest/miniconda.html) to manage your virtual environments. Miniconda is a lightweight version of [Anaconda](https://www.anaconda.com/), a popular Python distribution that comes with many of the packages that are commonly used in data science.

#### Instructions for setting up your environment using Miniconda:
1. If you do not already have Miniconda installed, download and install the latest version for your opperating system from the following link: [https://docs.conda.io/en/latest/miniconda.html#latest-miniconda-installer-links](https://docs.conda.io/en/latest/miniconda.html#latest-miniconda-installer-links)

2. Create a new virtual environment for this assignment by running the following command in your terminal:

   ```bash
   conda env create -f environment.yml
   ```

   This will create a new virtual environment called `biomedin215`

3. Activate your new virtual environment by running the following command in your terminal:

   ```bash
   conda activate biomedin215
   ```

   This will activate the virtual environment you created in the previous step.

4. Finally, ensure that your `ipynb` (this notebook)'s kernel is set to utilize
the `biomedin215` virtual environment you created in the previous steps. Depending on
which IDE you are using to run this notebook, the steps to do this may vary.


In [ ]:
# Run this cell: 
# The lines below will instruct jupyter to reload imported modules before 
# executing code cells. This enables you to quickly iterate and test revisions
# to your code without having to restart the kernel and reload all of your 
# modules each time you make a code change in a separate python file.

%load_ext autoreload
%autoreload 2

In [ ]:
# Run this cell to ensure the environment is setup properly
import pandas as pd
import numpy as np
import os
import warnings

print("Sanity check: Success")

### *Note to Students:* 📚

>Throughout the assignment, we have provided `sanity checks`: small warnings that will alert you when your implementation is different from the solution. Our goal in providing these numbers is to help you find bugs or errors in your code that may otherwise have gone unnoticed. Please note: the sanity checks are just tools we provided to be helpful, and should not be treated as a target to hit. We manually grade each assignment based on the code you submit, and not based on whether you get the exact same numbers as the sanity checks.

If you are failing the sanity checks (even by a lot) and your implementation is correct with minor errors, you will still receive the majority of the points (if not all the points).

In [ ]:
# Run this cell to set up sanity checks warnings
# Note: You do not need to change anything in this cell

# Creates a custom warning class for sanity checks
class SanityCheck(Warning):
    pass

# Sets up a cosutom warning formatter
def custom_format_warning(message, category, filename, lineno, line=None):
    if category == SanityCheck:
        # Creates a custom warning with orange text
        return f'\033[38;5;208mSanity Check - Difference Flagged:\n{message}\033[0m\n'
    
    return '{}:{}: {}: {}\n'.format(filename, lineno, category.__name__, message)

# Sets the warning formatter for the entire notebook
warnings.formatwarning = custom_format_warning

---

### Data Description 📂

We will be utilizing the same subset of the [MIMIC III database](https://mimic.mit.edu/docs/iii/about/) we utilized starting in A2: the 1,000 subject development cohort you created previously. You will start with a dataset very similar to what you may have generated at the end of the prior assignment.

You will analyze the available data to identify a cohort of patients that underwent septic shock during their admission to the ICU. **All of the data you need for this assignment is available on Canvas.** 

Once you have downloaded and unzipped the data, you should see the following `3` csv files:
- `patient_feature_matrix.csv`

- `cohort.csv`

- `feature_descriptions.csv`

**Specify the location of the folder containing the data in the following cells:**

In [ ]:
# Specify the path to the folder containing the data files
data_dir = "../../215data/A5_data/" # <-- TODO: You will need to change this path

In [ ]:
# Run this cell to make sure all of the files are in the specified folder
expected_file_list = ["patient_feature_matrix.csv", "cohort.csv", "feature_descriptions.csv"]

for file in expected_file_list:
    assert os.path.exists(os.path.join(data_dir, file)), "Can't find file {}".format(file)

print("All files found!")

In [ ]:
# Run this cell to load the data from the CSV files into Pandas DataFrames
patient_feature_matrix = pd.read_csv(os.path.join(data_dir, "patient_feature_matrix.csv"))
cohort = pd.read_csv(os.path.join(data_dir, "cohort.csv"))
feature_descriptions = pd.read_csv(os.path.join(data_dir, "feature_descriptions.csv"))

##### `Important note to students:`
You may notice that some of the various features appear to have multiple versions (ex. `icd9_AORTIC_DISSECTION_A` and `icd9_AORTIC_DISSECTION_B`). For the purposes of this assignment, we have done some simplification in preprocessing the data to make it easier for you to work with. In the future, it is important to keep in mind that healthcare datasets are notoriusly messy and often require a lot of preprocessing before they are ready to be used for analysis.

In [ ]:
# Run this cell to see the first three rows of the patient feature matrix
print("Patient Feature Matrix")
display(patient_feature_matrix.head(3))

In [ ]:
# Run this cell to see the first three rows of the cohort DataFrame
print("Cohort")
display(cohort.head(3))

In [ ]:
# Run this cell to see the first three rows of the feature descriptions DataFrame
print("Feature Descriptions")
display(feature_descriptions.head(3))

In [ ]:
# (OPTIONAL TODO:) It is always a good idea to filter out columns that you don't need from DataFrames.
# As always, feel free to add code to your notebooks to do this. This is not required for the assignment.
# You may want to come back to this later when you are more familiar with the data and know which columns you need.

# `1` (`5 points`) Preprocessing

## `1.1`: (`1 points`) Create the Feature Matrix and Outcome Vector

As you may have noticed above, the `death_in_stay` labels and features are all included together inside of the `patient_feature_matrix.csv` file. Let's split the patient_feature_matrix up into two numpy arrays.

Implement the `split_labels_and_features` function in `preprocessing.py`. When you are finished, run the following cell.

In [ ]:
from src.preprocessing import split_labels_and_features

# Run this cell to split the labels from the features
features, labels = split_labels_and_features(patient_feature_matrix,
                                             {"oxy_drop": {"stable": 0, "oxy_drop":1},
                                              "death_in_stay": {"survived": 0, "died": 1},
                                              "gender": {"M":0, "F": 1}})

# Sanity Check shapes
if features.shape[0] != labels.shape[0]:
    warnings.warn("Number of rows in features and labels don't match", SanityCheck)

if features.shape[1] != patient_feature_matrix.shape[1] - 2:
    warnings.warn("Number of columns in features is incorrect", SanityCheck)

if labels.shape[1] != 1:
    warnings.warn("Number of columns in labels is incorrect", SanityCheck)

# Sanity Check indices
if not features.index.name == "subject_id":
    warnings.warn("Index name is incorrect", SanityCheck)

if not labels.index.name == "subject_id":
    warnings.warn("Index name is incorrect", SanityCheck)

##### `Important note to students:`

It's worth noting that the definition of `gender` in the `MIMIC-III` database and many other clinical databases is a simplified representation of gender that does not account for a range of gender identities and expressions, and most often isn't distinguished from biological sex which may or may not reflect an individual's self-identified gender identity (SIGI). Increasingly, EHR schemas are updated/designed to capture SIGI in addition to biological sex to improve care for transgender and gender nonconforming (TGNC) patients.

As you continue your journey in medical data science, remember that it is critically important to always review the schema of the dataset you are working with to ensure that you know how different data fields are defined, so that you can fully understand the assumptions and limitations of your downstream analysis.

## `1.2` (`4 points`) Removing Uninformative Features

Before we do any modeling, let's cut down on our feature space by removing `low-variance features` that probably aren't useful enough to measure association with or use in a predictive model. 

Implement the function `feature_variance_threshold` in `preprocessing.py`. When you are finished, run the following cell.

In [ ]:
from src.preprocessing import feature_variance_threshold

# Run this cell to remove features with low variance
filtered_features = feature_variance_threshold(features, freq_cut=95/5, unique_cut=0.1)

**Report how many of each different kind of feature are left after filtering out the near-zero variance features. As a sanity check, look at the kinds of features that are over-represented or under-represented in this set relative to the full set of features. Provide a brief plausible explanation of what you observe.**

In [ ]:
# TODO: Use this cell to implement any code needed to answer this writen question
# HINT: It may be helpful to create a dataframe that contains the feature_type
# of the features in the features and filtered_features DataFrames by merging with
# the feature_descriptions DataFrame.
# This will allow you to easily compare which type of features were removed.

<span style="color:blue;"> YOUR ANSWER HERE </span>

# `2` (`55 points`) Associative Analyses

Associative analysis revolves around the concept of finding associations or patterns between features and outcomes. In biomedical data science, this approach can uncover hidden relationships between different medical conditions, drugs, genetic variations, or other biomedical entities, which can prove invaluable in research and clinical decision-making.

It's important to keep in mind that data science can help us find patterns, but we must always approach our results critically, as considering the broader clinical and biological context is essential to understanding the significance of our findings. Let's try to find some interesting associations in our dataset!

## `2.1` (`12 points`) Hypothesis Testing

Hypothesis testing is a cornerstone of statistical inference, allowing researchers to determine if there's enough evidence in a sample of data to infer that a particular condition holds for a larger population. In biomedical data science, this method is essential for validating the significance of findings, be it in clinical trials, genomic studies, or other investigations. If you are unfamiliar with hypothesis testing, we recommend that you review the following video: [StatsQuest - Hypothesis Testing](https://youtu.be/0oc49DyA3hU)

### Types of Hypothesis Tests
#### Tests for one random variable
- Continuous
    **One-sample t-test:**: If you want to test if the mean of a single sample is different from a specified value (usually a known mean).
- Discrete
    **Chi-squared ($\chi^2$) test:** Tests if the proportion of categorical outcomes is equal to a specified value.

#### Tests for two random variables
- One continuous variable vs one discrete variable with 2 levels $^*$

    - **For independent samples: Two-sample t-test:** *aka independent t-test*, used to determine whether there is a statistically significant difference between the means of two independent groups.

    - **For independent but small samples: Wilcoxon-Mann-Whitney test:** used to determine whether two independent samples come from the same distribution. (non-parametric alternative to the two-sample t-test)

    - **For paired samples: Paired t-test:** *aka dependent t-test*, used to determine if there is a statistically significant mean difference between two sets of paired or related observations.

    - **For paired but small samples: Wilcoxon signed-rank test:** used to determine whether two dependent samples come from the same distribution. (non-parametric alternative to the paired t-test)

    
- One continuous variable vs one discrete variable with > 2 levels $^*$
    - **One-way ANOVA**: used to determine whether there are any statistically significant differences between the means of three or more independent (unrelated) groups.
    
    <br>

- One discrete variable vs one discrete variable
    - **For r-level vs c-level: r x c contingency table Chi-squared test for independence:** used to determine if there's an association or relationship between two categorical variables where one variable has r levels and the other has c levels.

    - **For 2-level vs 2-level but small counts: Fisher's exact test:** similar to the Chi-squared test for a 2x2 contingency table, but used when the sample sizes are small (usually when the expected count/frequency in any cell of the table is less than 5)
    
    - **For paired samples - McNemar's test:** used for 2x2 contingency tables with paired or matched data. A classic example is when patients are observed before and after a treatment, and the response is dichotomous (e.g., "improved" or "not improved").

- One continuous variable vs one continuous variable:
    - **Correlation/simple linear regression:** used to determine if there is a relationship between two continuous variables.

#### Tests for more than two random variables
- One discrete vs one discrete vs one continuous:
    - **Two-way ANOVA:** used to determine whether there are any statistically significant differences between the means of three or more independent (unrelated) groups that have been split on two factors (i.e., independent variables).
    
    <br>

- One continuous vs many - **multiple linear regression**

- One discrete vs many - **logistic regression**



$^*$ Note: In statistics, the term `level` when referring to a discrete variable typically means a distinct value or category that the variable can assume.


> Credit: The above write up was adapted for this assignment from Lexin Li's 2016 Big Data course at the University of California Berkeley.

### `2.1.1` (`10 points`) Statistical Tests of Differences Between Two Groups

**For the each of the following features, use a `t-test`, `rank-sum test`, `Fisher exact test`, or a `Chi-squared test` (whichever is appropriate) to determine if there is a statistically significant association between the feature and the `death_in_stay` outcome. Write your reasoning for determining which kind of test to use. If multiple tests are applicable to a comparison, choose one and explain why you chose it.**

> NOTE: Make sure you read the documentation for the test functions. Statistical testing functions can be counter intuitive to use, and may produce and incorrect result if used incorrectly.

> Hint: It may be helpful to first create two new dataframes, one of patients who died in the ICU and one of patients who did not die in the ICU. Remember that you can add additional code cells to this notebook if needed.

In [ ]:
# Run this cell to import the test functions.
# Utilize one of the following imported functions in your answers below:
from scipy.stats import ttest_ind, fisher_exact, chi2_contingency, mannwhitneyu

# NOTE: for Rank Sum Tests use mannwhitneyu

# NOTE: For some of the functions above, you can pass series of labels and features. Others
# require you to pass the tables of counts. You may find the function pd.crosstab() useful for
# certain functions.

#### `alarms` (chartindicator_1622)

In [ ]:
# TODO: Implement the test here. Display both the test statistic and p-value

**Which test did you utilize for the `alarms` feature? Why?**

<span style="color:blue;"> YOUR ANSWER HERE </span>

#### `activity` (chartindicator_31)

In [ ]:
# TODO: Implement the test here. Display both the test statistic (or the odds ratio) and p-value

**Which test did you utilize for the `activity` feature? Why?**

<span style="color:blue;"> YOUR ANSWER HERE </span>

#### `respiratory rate` (chartvalue_618)

In [ ]:
# TODO: Implement the test here. Display both the test statistic and p-value

**Which test did you utilize for the `respiratory rate` feature? Why?**


<span style="color:blue;"> YOUR ANSWER HERE </span>

#### `Arterial PaCO2` (chartvalue_778)

In [ ]:
# TODO: Implement the test here. Display both the test statistic and p-value

**Which test did you utilize for the `arterial PaCO2` feature? Why?**


<span style="color:blue;"> YOUR ANSWER HERE </span>

#### `Oxygen Desaturation` (oxy_drop)

In [ ]:
# TODO: Implement the test here. Display both the test statistic and p-value

**Which test did you utilize for the `oxy_drop` feature? Why?**
**Was the association statistically significant?**

<span style="color:blue;"> YOUR ANSWER HERE </span>

#### `snomed ct concept C2720507` (C2720507)

In [ ]:
# TODO: Implement the test here. Display both the test statistic and p-value

**Which test did you utilize for the `C2720507` feature? Why?**


<span style="color:blue;"> YOUR ANSWER HERE </span>

### `2.1.2` (`2 points`) Hypothesis Testing with the Bonferroni Correction

Generally, when we perform hypothesis tests, we are attempting to determine if a particular effect or difference is statistically significant. We usually use a significance level, often denoted as α (alpha), to make this determination. A common choice for α is 0.05, meaning that there is a 5% chance of incorrectly rejecting the null hypothesis (a Type I error) when it is actually true.

However, when we conduct multiple hypothesis tests simultaneously – for example, when we are testing multiple variables or conditions at once – the likelihood of encountering at least one Type I error (i.e., finding at least one "significant" result when there actually is none) increases with the number of tests. This phenomenon is due to the accumulated risk from each individual test.

To address this problem, the Bonferroni correction is often employed. The Bonferroni correction is a simple method to control the familywise error rate (FWER), which is the probability of making one or more Type I errors in a set of comparisons.

To apply the correction, simply divide the desired α level by the number of tests being performed. For example, if we are performing 20 tests and want to maintain an overall α level (significance threshold) of 0.05, we would use an adjusted α level of 0.05/20 = 0.0025 for each individual test.

$\alpha_{adjusted} = \frac{\alpha_{desired}}{n_{tests}}$

We should note that the Bonferroni correction is a very conservative method, meaning that it is likely to produce false negatives (i.e., not catching real associations because we fail to reject the null hypothesis when it is actually false). However, it is a simple and effective way to control the familywise error rate (FWER), and is often used as a baseline for comparison with other methods.

Implement the function `calc_bonferroni` and `multi_ttest` in `associative_analysis.py`. When you are finished, run the following cell.

In [ ]:
from src.hypothesis import multi_ttest

multi_test_results, sig_columns = multi_ttest(filtered_features, labels, column_substring="chartvalue", alpha=0.05)

**How many of the chartvalue features are statistically significant after applying the Bonferroni correction?**

In [ ]:
# TODO: Use this cell the answer the written question above

<span style="color:blue;"> YOUR ANSWER HERE </span>

## `2.2` (`20 points`) Regression Analysis

Regression analyses model the relationship between a dependent variable (often denoted as Y) and an independent variable (often denoted as X). In biomedical data science, regression analyses are often used to predict a patient's risk of developing a particular condition or experiencing a particular outcome based on their clinical data. Let's take a look at some of the simpliest forms of regression analysis: linear regression.

Simple Linear Regression: Involves one dependent and one independent variable. The relationship is modeled as a linear equation, where the estimated dependent variable (Y) is a linear combination of the independent variable (X) and a constant known as the intercept.

$Y = \beta_0 + \beta_1 X + \epsilon$

Here, $\beta_0$ is the intercept, $\beta_1$ is the coefficient for X, and $\epsilon$ is the error term. The error term accounts for the variability in Y that cannot be explained by the linear relationship with X. The goal of linear regression is to find the values of $\beta_0$ and $\beta_1$ that minimize the error term. Finding the values of $\beta_0$ and $\beta_1$ that minimize the error term is called "fitting the model" or "training the model". Once we have a fit model, we can use it to make predictions for Y given new (or unseen) values of X. The estimated value of Y (denoted as $\hat{Y}$) for any given value of X can be calculated using the following equation:

$\hat{Y} = \beta_0 + \beta_1 X$

You can also have multiple independent variables in a linear regression model. This is known as multiple linear regression. The equation for multiple linear regression is:

$Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + ... + \beta_n X_n + \epsilon$

Where $\beta_0$ is the intercept, $\beta_1$ to $\beta_n$ are the coefficients for the independent variables $X_1$ to $X_n$, and $\epsilon$ is the error term.



Another type of regression is called `Logistic Regression`. Logistic regression is generally utilized when the dependent variable (the one we are trying to predict) is a binary outcome (e.g., "yes" or "no", "died" or "survived"). Logistic regression is similar to linear regression, but instead of predicting a continuous value, it predicts a value that falls between 0 and 1, which can be interpreted as a *predicted* probability of the dependent variable taking on the value 1 (or "yes", or "died", etc.). The equation for logistic regression is:

$Y = \frac{1}{1 + e^{-(\beta_0 + \beta_1 X_1 + \beta_2 X_2 + ... + \beta_n X_n + \epsilon)}}$

Where $\beta_0$ is the intercept, $\beta_1$ to $\beta_n$ are the coefficients for the independent variables $X_1$ to $X_n$, and $\epsilon$ is the error term.

**Main takeaway:**
- **Linear regression**: used to predict a **continuous** value (think: problems where you want to predict a value)

- **Logistic Regression**: used to predict a **binary** value (think: binary classification problems)


**In this part of the assignment you will build and compare several regression models to predict the binary outcome of `death_in_stay`, utilizing various features.**

### `2.2.1` (`6 points`) Regression models for association

Implement the `run_logistic_regression_model` function in `regression.py` following the implementation instructions in the docstring. When you are finished, run the following cell.


In [ ]:
from src.regression import run_logistic_regression

# glm_1: A logistic regression that only utilizes the "oxy_drop" and "age_in_days" features
model_1_features = ["oxy_drop", "age_in_days"]
glm_1 = run_logistic_regression(filtered_features, labels, model_1_features)

# glm_2: A logistic regression that utilizes the "oxy_drop", "age_in_days", and "gender" features
model_2_features = ["oxy_drop", "age_in_days", "gender"]
glm_2 = run_logistic_regression(filtered_features, labels, model_2_features)

# glm_3: A logistic regression that utilizes `age_in_days`, `gender`, `oxy_drop`, 
# and all of the chart value features that were found to have statistically significant
# associations with the outcome in the previous section.
model_3_features = ["oxy_drop", "age_in_days", "gender"]
model_3_features.extend(sig_columns)
glm_3 = run_logistic_regression(filtered_features, labels, model_3_features)

### `2.2.2` (`6 points`) Comparing regression models

Now that we have fit the models, let's compare them! 


**What is the point estimate and confidence interval for the `oxy_drop` coefficient in each model?**

**Why do you think the point estimate of the coefficient changes as we add more features to the model?**

> HINT: Use the result summaries from above to answer this question.


In [ ]:
# TODO: Use this cell to answer the written question above

<span style="color:blue;"> YOUR ANSWER HERE </span>

**Assuming you had a model of $Y$ regressed on $X_1$ and you added the variable $X_2$, under what conditions would the coefficient for $X_1$ _not_ change?**

<span style="color:blue;"> YOUR ANSWER HERE </span>

**If you had a model to estimate $Y$ regressed on $X_1$, and you added the variable $X_2$, what would likely happen to the coefficient of $X_1$ if both $X_1$ and $X_2$ were positively correlated with the outcome?**

<span style="color:blue;"> YOUR ANSWER HERE </span>

### `2.2.3` (`8 points`) Mean-Squared-Error (MSE)

Another way to compare models is to compare metrics such as the Mean Squared Error (MSE) and R-squared. These metrics are often used to evaluate the performance of regression models.

MSE is the average of the squared errors (the difference between the predicted value of Y and the actual value of Y). It is calculated as follows:

$MSE = \frac{1}{n} \sum_{i=1}^{n} (Y_i - \hat{Y_i})^2$


R-squared (also known as the coefficient of determination) represents the proportion of variance in the dependent variable that is explained by the independent variables in the model.

In [ ]:
model_1_probs = glm_1.predict(filtered_features[model_1_features])

In [ ]:
model_1_probs

In [ ]:
# Run this cell to get the R^2 and MSE values for each model
from sklearn.metrics import r2_score, mean_squared_error

# Let's compare these values for the different models
# First, let's get the predicted probabilities for each model
model_1_probs = glm_1.predict(filtered_features[model_1_features]).values
model_2_probs = glm_2.predict(filtered_features[model_2_features]).values
model_3_probs = glm_3.predict(filtered_features[model_3_features]).values

# Now, let's calculate the R^2 and MSE for each model
model_1_r2 = r2_score(labels, model_1_probs)
model_1_mse = mean_squared_error(labels, model_1_probs)
model_2_r2 = r2_score(labels, model_2_probs)
model_2_mse = mean_squared_error(labels, model_2_probs)
model_3_r2 = r2_score(labels, model_3_probs)
model_3_mse = mean_squared_error(labels, model_3_probs)

# Let's print these out
print(f"Model 1: R^2 = {model_1_r2}, MSE = {model_1_mse}")
print(f"Model 2: R^2 = {model_2_r2}, MSE = {model_2_mse}")
print(f"Model 3: R^2 = {model_3_r2}, MSE = {model_3_mse}")

**Which of the three models do you think has the best fit to the data we utilized? Explain your reasoning.**

<span style="color:blue;"> YOUR ANSWER HERE </span>

**If we were to use these models to predict outcomes for a group of new patients (patients the models had not seen before from a different dataset), would the same model perform the best on the new dataset as well? Explain your reasoning**

<span style="color:blue;"> YOUR ANSWER HERE </span>

## `2.3` (`23 points`) Survival Analysis

Survival analysis, also known as time-to-event analysis, is a branch of statistics that deals with analyzing the expected duration of time until one or more events happen. In the context of biomedical data science, these events often represent the occurrence of a particular health-related outcome, such as the onset of a disease, death, or recovery. Survival analysis is a powerful tool that can be used to answer a wide range of application areas, such as in clinical trials (to compare distributions of various treatment groups), epidemilogical studies (to identify risk factors that may influence the time to event prediction), and even patient monitoring (to predict the time to a equipment failure or a patient's discharge).

In this part of the assignment, you will perform some survival analysis on the cohort's data.

### `2.3.1` (`3 points`) Creating Survival Data

The first step of doing a survival analysis is to get our data into the appropriate format. To start, let's calculate the `survival time` for each patient. The `survival time` is the number of days between the patient's admission and the time of a particular event.

Implement the function `calc_survival_time` in `survival.py`. When you are finished, run the following cell.

In [ ]:
from src.survival import calc_survival_time

# Run this cell to calculate the survival time for each patient
survival_df = calc_survival_time(cohort)


# Sanity Check
if max(survival_df["survival_time_days"]) != 172:
    warnings.warn("Max survival time is incorrect", SanityCheck)

### `2.3.2` (`13 points`) Kaplan-Meier Curves

In survival analysis, an extremely commonly utilized tool is the `Kaplan-Meier (KM) estimator`, and the associated visualization method, the `Kaplan-Meier curve`. The `Kaplan-Meier (KM) estimator` is a non-parametric* method used to estimate the `survival function` from time-to-event data, which is the probability that a particular event has not occurred yet at a given time point. The `Kaplan-Meier curve` is a graphical representation of the `survival function` over time, and is often used to compare the survival distributions of different groups.

In this part of the assignment, you will set up a function to generate a `Kaplan-Meier curve` for the cohort's data. You will then use the `Kaplan-Meier curve` to compare the survival distributions of patients who experienced a sudden drop in oxygenation (oxy_drop) to those who did not. Before we get started, let's learn a bit more about how the `Kaplan-Meier estimator` works.

The `Kaplan-Meier estimator` is a product-limit estimator, meaning that it is calculated as the product of the conditional probabilities of surviving each time interval. At each time point in our data, we need to know the following:

- $n_i$ = The number of patients who are at risk of experiencing the event at time $t_i$
- $d_i$ = The number of patients who experienced the event at time $t_i$

The `Kaplan-Meier estimator` for the survival function at time $t_i$ is calculated as the product of the probabilities for all observed time points up to and including $t_i$:

$\hat{S}(t_i) = \prod_{j: t_j \leq t_i} \frac{n_j - d_j}{n_j} = \prod_{j: t_j \leq t_i} P(T > t_j | T > t_{j-1})$

The `Kaplan-Meier curve` is a graphical representation of the `Kaplan-Meier estimator` over time. The `Kaplan-Meier curve` is a step function, where the value of the curve at each time point is equal to the value of the `Kaplan-Meier estimator` at that time point (with time on the x-axis and the `Kaplan-Meier estimator` on the y-axis).

> \* A "non-parameteric" method in statistics refers to a type of method that makes few (if any) assumptions about data. This contrasts with "parametric" methods which assume some sort of distributional form for the data. The Kaplan-Meier curve is a non-parametric method because it does not assume any particular distribution for the survival times, and **utilizes only the observed data to estimate the survival function.**

Implement the function `display_kaplan_meier_curve` in `survival.py`. When you are finished, run the following cell.

In [ ]:
from src.survival import display_kaplan_meier_curve

display_kaplan_meier_curve(survival_df, "kaplan_meier_curve.png")

### `2.3.3` (`7 points`) Cox Proportional Hazards Model

Another fundamental tool of survival analysis is the `Cox Proportional Hazards Model`. The `Cox Proportional Hazards Model` is a semi-parametric* method used to model the `hazard function` from time-to-event data. The `hazard function` is the probability that a particular event occurs at a given time point, given that the event has not occurred yet. (Contrast the hazard function with the survival function we talked about above!)

The `Cox Proportional Hazards Model` is a popular tool for survival analysis because it is relatively simple to implement, and it allows us to estimate the effect of multiple variables on the `hazard function` simultaneously.

> \* A semi-parametric method is a method that includes both parametric and non-parametric components.

Here we will implement a Cox Proportional Hazards Model utilizing the `lifelines` library.

In [ ]:
# Run this cell to import the CoxPHFitter class from the lifelines package.
# Use this class to fit a Cox Proportional Hazards model to the survival data.
from lifelines import CoxPHFitter

In [ ]:
# Run this cell (no edits needed) to prepare the data for the cox model

# Create a survival dataframe that contains the survival time and feature columns
surv_features = filtered_features.merge(survival_df.drop(columns=["oxy_drop"]), on="subject_id").drop(columns=["subject_id"])

# Create a binary column for death event
surv_features["death_event"] = (surv_features['death_in_stay'] == 'died').astype(int)

# Remove the "death_in_stay" column
surv_features.drop(columns=["death_in_stay"], inplace=True)

Fit a Cox Proportional Hazards model using the cell below. You will need to use the `CoxPHFitter` class from the `lifelines` library. You can find the documentation for the `CoxPHFitter` class here: [https://lifelines.readthedocs.io/en/latest/fitters/regression/CoxPHFitter.html](https://lifelines.readthedocs.io/en/latest/fitters/regression/CoxPHFitter.html)



In your implementation, use `oxy_drop` as the only predictor(formula), `death_event` as the event, and `survival_time_days` as the duration.

In [ ]:
# TODO: Fit a Cox Proportional Hazards model to the survival data

**What is the point estimate and 95% confidence interval of the coefficient for `oxy_drop`?**



In [ ]:
# TODO: Use this cell to answer the written question (if needed)

<span style="color:blue;"> YOUR ANSWER HERE </span>

In addition, let's run another version of the model adjusted for all of the `sig_columns` (the columns known to have significant associations in `filtered_features`) AND the `oxy_drop` column. Use the same event and duration as above, and utilize the `sig_columns` and `oxy_drop` as the predictors (formula).



In [ ]:
# TODO: Implement the model here

**What is the point estimate and 95% confidence interval of the coefficient for `oxy_drop` in the adjusted model?**

In [ ]:
# TODO: Use this cell to answer the written question above (if needed)

<span style="color:blue;"> YOUR ANSWER HERE </span>

# `3` (`40 points`) Causal Analyses

In our associative analyses, we saw that we could do a reasonable job of predicting mortality, but there are many cases where having a prediction isn't as useful as understanding the underlying causal mechanisms. To improve standards of care, clinicians need to know which factors have causal relationships with mortality, so they can intervene correctly. This line of inquiry is known as **causal inference**, and it is a very active area of research in biomedical data science.

In this part of the assignment, we will explore some of the methods that can be used to infer causality from observational data. We will investigate to what extent having a sudden and sustained drop in oxygenation during ventilation (which we will refer to as the "exposure") is *causally* related to death later in the hospitalization (which we will refer to as the "outcome").

## `3.1` (`10 points`) Analyses without "Matching"

When someone refers to analyses without 'matching' in the context of causal inference, they are discussing a method of analysis where units (e.g., individuals in a study) are *not* paired or grouped based on similar characteristics or treatment propensities before estimating treatment effects. 

For this first part, let's see what kind of analysis we can do **without** matching.


### `3.1.1` (`3 points`) Univariate analysis

Univariate analysis, in a general statistical context, refers to the examination of a single variable's properties, distribution, and characteristics. However, in the context of causal inference, "univariate analysis" can be somewhat of a misnomer, given that causal inference inherently involves understanding the relationship between two or more variables: a treatment and an outcome, at minimum. 

When people refer to "univariate analysis" in the context of causal inference, they often mean examining the causal effect of one treatment on an outcome, without adjusting for potential confounders or covariates. In other words, it's a simple analysis that doesn't account for other variables that might influence the relationship between treatment and the outcome. 

Researchers often start with a univariate analysis to get a basic understanding of the relationship between variables, then proceed to multivariate analysis where they adjust for confounders and possibly investigate interactions between variables. For our univariate analysis in this context, let's run a statistical test to see if a drop in oxygenation is related to mortality.

**Run an appropriate statistical test to see if a drop in oxygenation is related to mortality. Treat both variables as binary.**


In [ ]:
# TODO: Implement the appropriate test of the options here in this cell
# HINT: You may have implemented a very similar test above
from scipy.stats import ttest_ind, fisher_exact, chi2_contingency, mannwhitneyu

**What is the odds ratio?**

<span style="color:blue;"> YOUR ANSWER HERE </span>

**Does a drop in oxygenation appear to significantly decrease or increase the risk of death? Does this establish causality between a drop in oxygenation and mortality? Why or why not? (If you think it does not, how else would you explain the result?)**

<span style="color:blue;"> YOUR ANSWER HERE </span>

### `3.1.2` (`5 points`) Multivariate analysis

Let's see if we still observe an effect after adjusting for other features.

In the previous section, we implemented a logisitic regression on different sets of features. Run the following cell to reuse your code from the previous section to fit a logistic regression model of mortality on the features that were found to have non-zero variance.


In [ ]:
# Run this cell (No changes necessary)
from src.regression import run_logistic_regression

glm_nzv = run_logistic_regression(filtered_features, labels, selected_features=None)

**Is the coefficient of the feature encoding a drop in oxygenation statistically significant in the model?**

In [ ]:
# TODO: Use this cell to answer the written question above (if needed)

<span style="color:blue;"> YOUR ANSWER HERE </span>

**What is the equivalent odds ratio?**

In [ ]:
# Use this cell to answer the written question above (if needed)

<span style="color:blue;"> YOUR ANSWER HERE </span>

**Does this establish causality between a drop in oxygenation and mortality? Why or why not? If you think it does not, how else would you explain the result?**

<span style="color:blue;"> YOUR ANSWER HERE </span>

### `3.1.3` (`2 points`) Can we do an experiment?

Usually questions related to causality are best answered through highly controlled expirements, such as Randomized Control Trials (RCTs). However, in many contexts in medicine and healthcare, it is either not possible or ethical to conduct an RCT.

Let's take our example: "To what extent is having a sudden and sustained drop in oxygenation during ventilation *causally* related to death later in the hospitalization"
- **Is there an experiment you could run that "in theory" would let you determine the causal effect of a sudden oxygenation drop on mortality using only these inferential analyses?** 

- **What might be some practical or ethical problems with your experiment?**

<span style="color:blue;"> YOUR ANSWER HERE </span>

## `3.2` (`30 points`) Matching Analysis with Propensity Scores

Let's see if we can more conclusively determine a causal effect of a sudden oxygenation drop on mortality using *matching* in our analysis. 

**Matching** is a technique used to control for confounding variables in observational data by grouping units (e.g. individuals in a study) based on similar characteristics or propensities before estimating treatment effects. Matching is a popular technique in causal inference because it allows us to estimate the causal effect of a treatment on an outcome by comparing units that are similar in all aspects except for the treatment. 

> Note that sometimes "matching" is described as mechanism to "mimic" or "emulate" the randomization of a RCT. This interpretation is incorrect: matching is just a tool you can use to adjust for confounders in a non-randomized study, but does not introduce randomization.

In this assignment, we will utilize `propensity scores` to perform a matching analysis. 

Let's start with the definition: A `propensity score` is the probability that a unit (e.g. an individual in a study) was exposed to a particular **`treatment or exposure`** given their observed characteristics. So in our example, the propensity score of an individual patient is the probability that they had a sudden oxygenation drop (`oxy_drop` is 1) given their other features.

A propensity score matching analysis involves the following steps:

1. Calculate the propensity scores for each unit in the dataset:
    The first step in our process is to determine propensity scores for each unit in the dataset. In our example, we want to calculate the probability that a patient had a sudden oxygenation drop (`oxy_drop` is 1) given their other features. We can do this by fitting a logistic regression model of `oxy_drop` on the other features in the dataset (you do not including the outcome: `death_in_stay`). The predicted probabilities from this model are the propensity scores for each patient.

2. Match units based on their propensity scores:
    There are multiple strategies to do this, but the idea is that patients who have similar propensity scores are likely to have similar characteristics, and therefore are likely to be comparable in terms of their potential outcomes. In our example, we will use a method called "Caliper Matching". We will discribe caliper matching in more detail in the sections below, and utilize it to match patients in the exposure group to patients in the control group (note that not all patients will be matched).

3. Run an analysis only utilizing patients that were matched in the previous step.

Patients who have similar propensities are more "twin-like" and therefore more comparable in terms of their potential outcomes. If two patients have similar propensity scores and one of them was "exposed" while the other was not, we can be more confident that any difference in their outcomes is due to the exposure, rather than due to differences in their characteristics.

### `3.2.1` (`5 points`) Propensity Modeling

Let's start our propensity analysis by calculating the propensity scores for each patient in the dataset. Utilizing the same code from before, lets fit a logisitic regression model to predict `oxy_drop` using the other features in the dataset.



In [ ]:
from src.regression import run_logistic_regression

# Pull the list of features from the filtered_features dataframe
feature_list = filtered_features.columns.tolist()

# Remove the oxy_drop feature from the list
feature_list.remove("oxy_drop")

# Run the logistic regression
glm_propensity = run_logistic_regression(filtered_features, filtered_features[["oxy_drop"]], selected_features=feature_list)

In [ ]:
# (No modifications needed in this cell)
# Run this cell to use the model to calculate propensity scores
propensity_scores = glm_propensity.predict(filtered_features[feature_list])

# Name the series "propensity_score"
propensity_scores.name = "propensity_score"

# Merge with oxy drop
propensity_scores = pd.concat([propensity_scores, filtered_features["oxy_drop"]], axis=1)

# Display the first 5 rows of the propensity_scores series
propensity_scores.head()

### `3.2.2` (`2 points`) Propensity Estimation

Now that we have fit our propensity model and have calculated our propensity scores, let's take a look at the distribution of propensity scores in our dataset. Rather than plotting the distribution of propensity scores themselves, we will plot the distirubtions of the logits of the propensity scores.

The **logit** of a probability is defined as the natural logarithm of the odds of the probability. The logit of a probability $p$ is calculated as follows:

$logit(p) = ln(\frac{p}{1-p})$

**Why are visualizing the logit transformation of the propensity scores, rather than the scores themselves?**
Transformations (like the logit transformation) are often applied to sets of data to "change the scale", which can be really helpful for visualizing or interpreting distributions. In this case, we are using the logit transformation to "re-scale" the propensity scores (which are bounded between 0 and 1) to a range that is unbounded. Propensity scores close to 0 suggest a low probability of exposure, while scores close to 1 suggest a high probability of exposure. In some datasets, many scores can cluster near these boundaries, making it hard to visually assess overlap in the distributions. After applying the logit transformation, a propensity score of 0.5 is centered at 0, giving a much more normalized distribution. By transforming to logits, the scores that were clustered near 0 and 1 are spread out more evenly, making it easier to see the distribution of both groups and assess the overlap.

The primary goal of propensity score methods in observational studies is to reduce confounding by making the treated and untreated groups comparable in terms of the observed covariates. If there's significant overlap in the logit-transformed scores, it indicates that there's a good chance of finding comparable treated and untreated observations for causal inference analysis.

Implement the function `plot_logit_scores` in `matching.py` and run the following cell to test your implementation.

In [ ]:
from src.matching import plot_logit_scores

# Run this cell to plot the logits of the propensity scores
plot_logit_scores(propensity_scores, "logit_score_plot.png")

**What does the amount of overlap between these two distributions say about how good your propensity score model is predicting the exposure?** 

**Is it bad if your model has poor performance? (hint: if the exposure happened totally at random, would that help you or hurt you in determining a causal effect?)**

<span style="color:blue;"> YOUR ANSWER HERE </span>

### `3.2.3` (`3 points`) Caliper Matching

The next step of our propensity score analysis is to match patients based on their propensities using Caliper Matching. Caliper Matching involves matching units (patients in this case) within a specified distance (caliper) of each other. The caliper is a threshold that defines the maximum distance between two units' propensity scores for them to be considered a match.


Implement the function `caliper_match` in `matching.py` using the instructions in the docstring. When you are finished, run the following cell.

In [ ]:
from src.matching import caliper_match

# Run this cell to perform the matching
match_list = caliper_match(propensity_scores, caliper = 0.25)

# Sanity Check
all_matched_subject_ids = [subject_id for match_pair in match_list for subject_id in match_pair]

if len(all_matched_subject_ids) != len(set(all_matched_subject_ids)):
    warnings.warn("Duplicate subject_id found in all_matched_subject_ids", SanityCheck)

**How many patients are in the matched set? How does this number compare to the original number of patients in the dataset?**


<span style="color:blue;"> YOUR ANSWER HERE </span>

In [ ]:
# TODO: Use this cell to answer the written question above (if needed)

### `3.2.4` (`5 points`) Matched Outcomes Analysis

Now let's repeat our univariate analysis, but this time only use data from the matched patients.

In [ ]:
# TODO: Implement the appropriate test of the options here in this cell
# HINT: You may have implemented a very similar test above
from scipy.stats import ttest_ind, fisher_exact, chi2_contingency, mannwhitneyu

**What is the p-value?**

**What is the test statistic OR odds ratio?**

<span style="color:blue;"> YOUR ANSWER HERE </span>

### `3.2.5` (`5 points`) Effect of Caliper Width

"Bias" and "Variance" are concepts that are integral to both machine learning and causal inference, though their implications can be somewhat different depending on the context. (NOTE: Both "Bias" and "Variance" are very overloaded terms! The definitions change depending on the context.)

In the context of discussing a "bias" vs "variance" tradeoff:

- **Bias** refers to the difference between the average prediction of our model and the correct value we are trying to predict. A model is said to have high bias if the model oversimplifies the "true" relationship between the predictors and outcome, generally referred to as "being underfit". Models with high bias often have systematic errors in predictions (errors largely occuring in the same direction with similar magnitude) as the estimates are "biased" away from the true values.

- **Variance** refers to the variability of our model's predictions. A model is said to have high variance if small changes to the training data result in large changes in the predictions. Models with high variance are often **overfit** to the training data, and do not generalize well to new data.



You may find this article from "Towards Data Science" helpful for exploring this concept further: [Understanding the Bias-Variance Tradeoff](https://towardsdatascience.com/understanding-the-bias-variance-tradeoff-165e6942b229)

In our propensity analysis, we could have had a stricter caliper and enforced that patients had to be more similar to be matched. 

**What would have been the tradeoff in terms of the **bias** and **variance** of utilizing a smaller caliper on our models predictions?**

<span style="color:blue;"> YOUR ANSWER HERE </span>

### `3.2.6` (`5 points`) Including Post-Exposure Data in Propensity Models

Think back to the previous assignment where we produced the dataset we are working with now. All of the information we have for each patient is from before the **index_time**, which by definition is before the **exposure_time**. 

Heres a hypothetical:
Your friend Marc wants to include the "total cost of the patient's hospitalization" as an additional feature in our dataset, and utilize it to fit our propensity model. After looking into his idea, you realize that the "total cost" feature is generated **after** the exposure_time for each patient. 

**What would be the problem with including this feature in our propensity model? Would our inference still be causally valid? Why or why not?**



<span style="color:blue;"> YOUR ANSWER HERE </span>

### `3.2.7` (`5 points`) Lingering Biases


What are some sources of **bias** (still taking about bias in the sense of "bias" vs "variance" tradeoff) that our propensity score analysis cannot correct for?

<span style="color:blue;"> YOUR ANSWER HERE </span>

# `4` (`10 points Extra Credit`) **Optional** Reflection Exercise 

Find a BIOMEDIN 215 classmate who completed Assignment 5 in `R`. 

You'll notice that the results of some of their statistical analyses are quite different from yours. Why do you think that is?

> Hint: there is a very specific design decision early on in the assignment that makes some results differ substantially between the two versions -- if you can find it, we'll give you 10 bonus points!

<span style="color:blue;"> YOUR ANSWER HERE </span>

---

# Feedback (`0 points`)
Please fill out the following [feedback form](https://docs.google.com/forms/d/e/1FAIpQLSevaufyQLf5HAFTStk15OJ5idA5OkdLDMsEp8v-fSoPlXKxow/viewform?usp=sf_link) so we can improve the course for future students!

---

# Submission Instructions ✅

There are two files you must submit for this assignment:

1. A `PDF` of this notebook.
- **Please clear any large cell outputs from executed code cells before creating the PDF.**
    - Including short printouts is fine, but please try to clear any large outputs such as dataframe printouts. This makes it easier for us to grade your assignments!
- To export the notebook to PDF, you may need to first create an HTML version, and then convert it to PDF.

2. A `zip` file containing your code generated by the provided `create_submission_zip.py` script:
- Open the `create_submission_zip.py` file and enter your SUNet ID where indicated.
- Run the script via `python create_submission_zip.py` to generate a file titled `<your_SUNetID>_submission_A5.zip` in the root project directory.
